# Comparing Inference Methods

This notebook is a stub for comparing the **power-spectrum** and **full-field**
inference approaches.

Currently it:
1. Loads posterior samples from a power-spectrum run (`samples_*.npz`)
2. Inspects the recovered power spectra and parameter distributions

Future additions:
- Load full-field posterior samples for side-by-side comparison
- Corner plots of cosmological parameters
- Comparison of posterior widths and biases

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import glob

import jax_cosmo as jc
import matplotlib.pyplot as plt
import numpy as np

import fwd_model_tools as ffi

## Load Power-Spectrum Posterior Samples

In [ ]:
PS_OUTPUT_PATH = "output/ps_inference"  # <- update to match notebook 12

npz_files = sorted(glob.glob(f"{PS_OUTPUT_PATH}/samples_*.npz"))
print(f"Found {len(npz_files)} batch file(s)")

all_samples: dict[str, list] = {}
for f in npz_files:
    data = np.load(f)
    for k in data.files:
        all_samples.setdefault(k, []).append(data[k])

samples = {k: np.concatenate(v, axis=0) for k, v in all_samples.items()}

print("\nLoaded sites:")
for k, v in samples.items():
    print(f"  {k}: shape={v.shape}")

## Posterior Parameter Distributions

In [ ]:
cosmo_keys = [k for k in samples if np.issubdtype(samples[k].dtype, np.floating) and samples[k].ndim == 1]

fig, axes = plt.subplots(1, len(cosmo_keys), figsize=(4 * len(cosmo_keys), 3))
if len(cosmo_keys) == 1:
    axes = [axes]

fiducial = jc.Planck18()

for ax, key in zip(axes, cosmo_keys):
    ax.hist(samples[key], bins=20, density=True, alpha=0.7, label="posterior")
    if hasattr(fiducial, key):
        ax.axvline(getattr(fiducial, key), color="red", linestyle="--", label="fiducial")
    ax.set_xlabel(key)
    ax.set_ylabel("density")
    ax.legend()

fig.suptitle("Power-spectrum posterior", y=1.02)
plt.tight_layout()
plt.show()

## Posterior Predictive Power Spectrum

Draw a sample of cosmologies from the posterior and overlay the predicted
C_ell against the observed spectrum.

In [ ]:
# Load the observed C_ell from notebook 12 output (re-run or pass in)
# obs_cl = ...  # shape (n_ell,) for the first auto-spectrum bin

# Placeholder: print summary statistics instead
print("Posterior summary:")
for key in cosmo_keys:
    v = samples[key]
    print(f"  {key}: mean={v.mean():.4f}  std={v.std():.4f}  "
          f"[{np.percentile(v, 16):.4f}, {np.percentile(v, 84):.4f}]")